# ==============================================
# 2_data_preprocessing.ipynb
# ==============================================

# Zelle 1: Bibliotheken importieren
import pandas as pd
import os
import re
import json
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

nltk.download('punkt')  # Falls nicht schon geschehen

# Definiere Pfade zum "raw" und "processed" Verzeichnis
RAW_DATA_DIR = "../data/raw"
PROCESSED_DATA_DIR = "../data/processed"

# ==============================================
# Zelle 2: Rohdaten laden
# ==============================================
# Beispiel: Wir haben eine Datei "fairytales_sample.csv" im RAW_DATA_DIR,
# die eine Spalte "text" mit kompletten Märchen oder Fantasy-Kapiteln enthält.

csv_file_path = os.path.join(RAW_DATA_DIR, "fairytales_sample.csv")
df = pd.read_csv(csv_file_path)

print(f"Anzahl Datensätze: {len(df)}")
display(df.head())

# ==============================================
# Zelle 3: Datensäuberung
# ==============================================
# Beispielhafte Funktionen, um Text zu bereinigen (HTML-Tags, Sonderzeichen etc.)

def clean_text(text):
    if not isinstance(text, str):
        text = str(text)
    # HTML-Tags entfernen
    text = re.sub(r"<.*?>", "", text)
    # Mehrfache Leerzeichen reduzieren
    text = re.sub(r"\s+", " ", text)
    # Optional: Lowercasing
    text = text.lower().strip()
    return text

df["cleaned_text"] = df["text"].apply(clean_text)

# (Optional) Entferne Einträge, die zu kurz sind (z. B. weniger als 50 Zeichen)
df = df[df["cleaned_text"].str.len() > 50]

print("\nBeispiel für gesäuberte Texte:")
display(df["cleaned_text"].head())

# ==============================================
# Zelle 4: Tokenisierung
# ==============================================
# Wir zerlegen jeden Text mithilfe von NLTK in Wort-Tokens (inkl. Satzzeichen).

def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

df["tokens"] = df["cleaned_text"].apply(tokenize_text)

print("\nBeispiel für Tokens:")
display(df["tokens"].head())

# ==============================================
# Zelle 5: Vokabular erstellen
# ==============================================
# Wir sammeln alle Tokens in einer Liste, zählen Frequenzen und
# wählen die häufigsten Tokens für das Vokabular (z. B. top N oder komplett).

from collections import Counter

all_tokens = []
for token_list in df["tokens"]:
    all_tokens.extend(token_list)

token_freqs = Counter(all_tokens)
print(f"Anzahl eindeutiger Tokens: {len(token_freqs)}")

# Beispiel: Beschränke das Vokabular auf die top-N häufigsten Tokens
VOCAB_SIZE = 20000  # oder len(token_freqs), wenn du alles behalten willst
most_common_tokens = token_freqs.most_common(VOCAB_SIZE)

# Reserviere Sondertokens
special_tokens = ["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
word2id = {}
idx = 0
for st in special_tokens:
    word2id[st] = idx
    idx += 1

for token, freq in most_common_tokens:
    if token not in word2id:  # Vermeide Doppler
        word2id[token] = idx
        idx += 1

print(f"Vokabulargröße (inkl. Sondertokens): {len(word2id)}")

# Inverses Mapping (für Generierung)
id2word = {v: k for k, v in word2id.items()}

# ==============================================
# Zelle 6: Tokens in IDs umwandeln
# ==============================================
# Ersetze jedes Token durch seine ID. Nicht gefundene Tokens -> <UNK>.

UNK_ID = word2id["<UNK>"]
BOS_ID = word2id["<BOS>"]
EOS_ID = word2id["<EOS>"]

def tokens_to_ids(token_list, word2id, unk_id=UNK_ID):
    return [word2id[t] if t in word2id else unk_id for t in token_list]

df["token_ids"] = df["tokens"].apply(lambda tk: tokens_to_ids(tk, word2id))

print("\nBeispiel für token_ids:")
display(df["token_ids"].head())

# ==============================================
# Zelle 7: Erstellung von Sequenzen & Splitting
# ==============================================
# Häufig macht man für ein LSTM-Sprachmodell: 
# input_seq = token_ids[:-1], target_seq = token_ids[1:] 
# Du kannst das aber auch erst im Dataset-Skript machen (dataset.py).
# Hier (zur Vereinfachung) speichern wir die gesamte token_ids-Liste.
# Wir teilen die Datensätze in train/val/test auf.

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# ==============================================
# Zelle 8: Speichern der Ergebnisse
# ==============================================
# 1) Vokabular -> vocab.json
import json

vocab_path = os.path.join(PROCESSED_DATA_DIR, "vocab.json")
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

with open(vocab_path, "w", encoding="utf-8") as f:
    json.dump(word2id, f, ensure_ascii=False)

print("\nVokabular gespeichert in:", vocab_path)

# 2) Train/Val/Test CSV
train_path = os.path.join(PROCESSED_DATA_DIR, "train.csv")
val_path = os.path.join(PROCESSED_DATA_DIR, "val.csv")
test_path = os.path.join(PROCESSED_DATA_DIR, "test.csv")

train_df.to_csv(train_path, index=False)
val_df.to_csv(val_path, index=False)
test_df.to_csv(test_path, index=False)

print("Train/Val/Test in CSV-Dateien gespeichert:")
print(train_path, "\n", val_path, "\n", test_path)

# ==============================================
# Zelle 9: Ausblick
# ==============================================
print("""
Die Daten liegen nun tokenisiert und aufgeteilt in train/val/test vor.
In '3_training_demo.ipynb' oder in 'src/train.py' kannst du nun dein LSTM
oder Transformer-Modell trainieren.
""")
